# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Name, UNI

Member 2: Name, UNI

Member 3: Name, UNI

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [2]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        self.status = 'train'
        self.X_val = None
        self.y_val = None
        
        # init parameters
        for i in range(1, self.num_layers):
            self.parameters[('W', i)] = np.random.normal(0, 1, (layer_dimensions[i], layer_dimensions[i - 1]))
            self.parameters[('W', i)] /= np.sqrt(layer_dimensions[i - 1])
            self.parameters[('b', i)] = np.zeros([layer_dimensions[i], 1])

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        M = None
        if self.status == 'train' and self.drop_prob > 0:
            # dropout only when training
            A, M = self.dropout(A, self.drop_prob)
        Z = np.dot(W, A) + b
        cache = (A, W, b, Z, M)
        return Z, cache

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        return self.relu(A)

    def relu(self, X):
        A = np.maximum(0, X)
        assert (X.shape == A.shape)
        return A
            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.uniform(size = A.shape)
        M = (M > prob) / (1 - prob)
        A = A * M
        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        
        cache = {}
        A = X
        for l in range(1, self.num_layers):
            Z, cache_l = self.affineForward(A,
                                            self.parameters[('W', l)], 
                                            self.parameters[('b', l)])
            cache[l] = cache_l
            A = self.activationForward(Z)
        # return AL, cache
        return A, cache
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        assert (self.status == 'train')
        # compute loss
        S = AL.shape[1]
        probs = np.exp(AL - np.max(AL, axis=0, keepdims=True))
        probs /= np.sum(probs, axis=0, keepdims=True)
        cost = -np.sum(np.log(probs[y, np.arange(S)])) / S
        
        if self.reg_lambda > 0:
            # add regularization
            for l in range(1, self.num_layers):
                cost += 0.5 * self.reg_lambda * np.sum(np.power(self.parameters[('W', l)], 2))
        dAL = probs.copy()
        dAL[y, np.arange(S)] -= 1
        dAL /= S
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
#        A = cache[0]
#        W = cache[1]
#        b = cache[2]
        A, W, b, Z, M = cache
        S = A.shape[1]
        
        dZ = self.activationBackward(dA_prev, cache)
        dA = np.dot(W.T, dZ)
        dW = np.dot(dZ, A.T) / S
        db = np.sum(dZ, axis = 1, keepdims=True) / S # or np.mean() ?
        # dA = W.T.dot(dA_prev) 
        # dW = dA_prev.dot(A.T) / S
        # db = np.sum(dA_pre, axis = 1) / S
        return dA, dW, db
    
    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        return self.relu_derivative(dA, cache[3]) # cache[3] == Z[l]
        
    def relu_derivative(self, dx, cached_x):
        out = np.maximum(0, cached_x)
        out[out > 0] = 1
        dx = out * dx
        return dx

    def dropout_backward(self, dA, cache):
        A, W, b, Z, M = cache
        return dA * M # cache[4] = M

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        assert (self.status == 'train')
        gradients = {}
        dA = dAL
        for l in range(self.num_layers-1, 0, -1):
            dA, dW, db = self.affineBackward(dA, cache[l])
            if self.drop_prob > 0:
                dA = self.dropout_backward(dA, cache[l])
            # assert (dW.shape == self.parameters[('W', l)].shape), '{} - {}'.format(dW.shape, self.parameters[('W', l)].shape)
            # assert (db.shape == self.parameters[('b', l)].shape), '{} - {}'.format(db.shape, self.parameters[('b', l)].shape)
            gradients[('W', l)] = dW
            # gradients[('b', l)] = db # should b be regularized ?
            if self.reg_lambda > 0:
                # add gradients from L2 regularization to each dW
                gradients[('W', l)] += self.reg_lambda * self.parameters[('W', l)]
               #  gradients[('b', l)] += reg_lambda * self.parameters[('b', l)]
        return gradients

    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for key in gradients.keys():
            self.parameters[key] -= alpha * gradients[key]
            
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        assert (alpha * self.reg_lambda < 1)
        self.status = 'train'
        self.parameters['mean'] = np.mean(X, axis = 1, keepdims = True)
        self.parameters['var'] = np.var(X, axis = 1, keepdims = True)
        X = (X - self.parameters['mean']) / np.sqrt(self.parameters['var'])
        for i in range(0, iters):
            # get minibatch
            X_batch, y_batch = self.get_batch(X, y, batch_size)
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)
            # compute loss
            cost, dAL = self.costFunction(AL, y_batch)
            # compute gradients
            gradients = self.backPropagation(dAL, y_batch, cache)
            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
            if i % print_every == 0:
                # print cost, train and validation set accuracies
                trn_acc = self.score(self.predict(X), y)
                if self.X_val is not None:
                    val_acc = self.score(self.predict(self.X_val), self.y_val)
                else:
                    val_acc = np.nan
                print('iter={:5}, cost={:.4f}, trn_acc={:.4f}, val_acc={:.4f}'.format(i, cost, trn_acc, val_acc))
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        self.status = 'predict'
        X = (X - self.parameters['mean']) / np.sqrt(self.parameters['var'])
        AL, _ = self.forwardPropagation(X)
        y_pred = np.argmax(AL, axis = 0)
        self.status = 'train'
        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        batch_idx = np.random.randint(X.shape[1], size = batch_size)
        X_batch = X[:, batch_idx]
        y_batch = y[batch_idx]
#         print y_batch.shape
        return X_batch, y_batch
    
    def score(self, y_pred, y_gold):
        return np.mean(y_pred == y_gold)
    
    def load_validation_set(self, X_val, y_val):
        self.X_val = X_val
        self.y_val = y_val

In [3]:
# Helper functions, DO NOT modify this
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [4]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [5]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [6]:
print(X_train.shape)
print(y_train.shape)
print(y_train[:5])

(3072, 50000)
(50000,)
[6 1 6 6 8]


In [42]:
y_train.reshape(-1).shape

(50000,)

In [ ]:
row_dim = x.shape[0]
col_dim = np.prod(x.shape[1:])
x_reshape = x.reshape(row_dim, col_dim)

## Part 1

#### Simple fully-connected deep neural network

In [8]:
layer_dimensions = [X_train.shape[0], 500, 100, 25, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=5000, alpha=1, batch_size=100, print_every=100)

AttributeError: module 'numpy' has no attribute 'range'

In [ ]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

# expression testing

In [46]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[-1, 2, 3], [4,5,6]])
c = np.array([1,2,3]) 

In [54]:
c.ravel().tolist()

[1, 2, 3]

In [47]:
a.T.dot(a)

array([[17, 22, 27],
       [22, 29, 36],
       [27, 36, 45]])

In [48]:
a = np.array([[1, 2, 3], [4, 5, 6]])

In [49]:
prob = np.exp(a - np.max(a, axis=0, keepdims=True)) 
prob

array([[ 0.04978707,  0.04978707,  0.04978707],
       [ 1.        ,  1.        ,  1.        ]])

In [59]:
np.arange(300).shape

(300,)

In [61]:
np.arange(3).shape

(3,)

In [60]:
prob[[0, 1, 1], np.arange(3)]

array([ 0.04978707,  1.        ,  1.        ])

AttributeError: 'list' object has no attribute 'shape'

In [31]:
np.sum(prob, axis=0, keepdims=True)

array([[ 1.04978707,  1.04978707,  1.04978707]])

In [44]:
N = a.shape[1]

In [43]:
np.arange(N)

array([0, 1, 2])

In [40]:
prob

array([[ 0.04978707,  0.04978707,  0.04978707],
       [ 1.        ,  1.        ,  1.        ]])

In [47]:
prob[[1, 1, 0], np.arange(N)]

array([ 1.        ,  1.        ,  0.04978707])

In [38]:
y_train.shape

(50000,)

In [32]:
x.shape

(2, 4, 5, 6)

In [34]:
row_dim = x.shape[0]
row_dim

2

In [36]:
col_dim = np.prod(x.shape[1:])
col_dim

120

In [30]:
weight_size

360

In [37]:
X_train.shape


(3072, 50000)

In [20]:
x.shape

(3072, 50000)

In [21]:
row_dim = x.shape[0]
row_dim

3072

In [23]:
x.shape[1:]

(50000,)

In [24]:
np.prod(x.shape[1:])

50000

In [22]:
col_dim = np.prod(x.shape[1:])
col_dim

50000

In [17]:
x_reshape = x.reshape(row_dim, col_dim)

In [18]:
x_reshape.shape

(3072, 50000)

In [51]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [53]:
np.sum(a, axis = 1)

array([ 6, 15])